<a href="https://colab.research.google.com/github/LogeshkumarTamilselvam/Phonepe_Plus/blob/main/Phonepe%20plus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile phone.py

import pandas as pd
import mysql.connector as sql
import streamlit as st
import plotly.express as px
import os
import json
from PIL import Image
from git.repo.base import Repo
from streamlit_option_menu import option_menu
from streamlit_lottie import st_lottie
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import plotly.graph_objects as go
import plotly.colors as colors

#Set the page config
st.set_page_config(
    page_title="Phonepe Plus ",
    page_icon=(":shark:"),
    layout="wide",
    initial_sidebar_state="expanded",
    menu_items={
        'Get help': 'https://github.com/LogeshkumarTamilselvam',
        'Report a bug': "https://www.linkedin.com/in/logeshkumar-tamilselvam-7258aa269/",
        'About': "# This is a header. This app only for *education purpose* created by Logeshkumar Tamilselvam!"
    }
)
# Load the image file using an absolute path
image_path = 'C:/Users/loges/Phonepeimage.jpeg'
with open(image_path, 'rb') as f:
    image_bytes = f.read()

# Display the image
st.image(image_bytes, use_column_width=True)
st.sidebar.header(":skull: :violet[**Hi folks! Choose the option option what you want**]")

#connect sql server
mydb = sql.connect(host="localhost",
                   user="root",
                   password="Logesh@123456789",
                   database= "phonepe"
                  )
print(mydb)
mycursor = mydb.cursor(buffered=True)

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

#define option menu
selected = option_menu("Main Menu", ["Home","Top Data","Explore Now"], 
                icons=["archive","check-square-fill","bar-chart-line", "exclamation-circle"],
                menu_icon= "menu-button-wide",
                default_index=0,
                orientation = "horizontal",
                styles={"nav-link": {"font-size": "20px", "text-align": "left", "margin": "-2px", "--hover-color": "#6F36AD"},
                        "nav-link-selected": {"background-color": "#6F36AD"}})
#user selected Home
if selected == "Home":
    with open("spotlight_1.jpg", 'rb') as f:
        image_bytes = f.read()

    with st.container():
        st.image(image_bytes, use_column_width=True)
        st.markdown("# :violet[PhonePe Plus - THE BEAT OF PROGRESS]👨‍💻")
        st.markdown("## :red[The site only for education purpose]⚠️")

    session = requests.Session()
    retry = Retry(
        total=3,
        backoff_factor=0.5,
        status_forcelist=[429, 500, 502, 503, 504],
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    lottie_url = "https://assets6.lottiefiles.com/packages/lf20_cmaqoazd.json"
    lottie_animation = session.get(lottie_url).json()

    lottie_width = 400
    lottie_height = 500

    column1, column2 = st.columns([1,1],gap = "small")
    
    
    with column1:
        st.write(" ")
        st.write(" ")
        st.markdown("### :violet[Description :] The Indian digital payments story has truly captured the world's imagination. From the largest towns to the remotest villages, there is a payments revolution being driven by the penetration of mobile phones and data.")
        st.markdown("### :violet[Source by :] [Phonepe](https://phonepe.com/)")
        st.markdown("### :violet[Social media:][Linkedin](https://www.linkedin.com/in/logeshkumar-tamilselvam-7258aa269/)")
        st.markdown("### :violet[Resource:][Github](https://github.com/LogeshkumarTamilselvam)")
    with column2:
        st_lottie(lottie_animation, speed=1, width=lottie_width, height=lottie_height, key="hello")
        
        
#user selected top datalist      
if selected == "Top Data":
    st.markdown("## :violet[Data Collections]")
    Year = st.slider("**Year**", min_value=2018, max_value=2022)
    Quarter = st.slider("Quarter", min_value=1, max_value=4)
    
    if st.selectbox("**Type**", ("Transactions", "Users")) == "Transactions":
        
        column1,column2 = st.columns([1,1],gap="small")
    
    with column1:
        st.markdown("### :violet[State]")
        mycursor.execute(f"select state, sum(Transaction_count) as Total_Transactions_Count, sum(Transaction_amount) as Total from aggregated_transaction where year = {Year} and quarter = {Quarter} group by state order by Total desc limit 10")
        df = pd.DataFrame(mycursor.fetchall(), columns=['State', 'Transactions_Count','Total_Amount'])
        fig = px.pie(df, values='Total_Amount',
                     names='State',
                     title='Top 10',
                     color_discrete_sequence=px.colors.sequential.Agsunset,
                     hover_data=['Transactions_Count'],
                     labels={'Transactions_Count':'Transactions_Count'})
        
        
        fig.update_traces(textposition='inside', textinfo='percent+label')
        st.plotly_chart(fig,use_container_width=True)
        
    with column2:
        st.markdown("### :violet[District]")
        mycursor.execute(f"select district , sum(Count) as Total_Count, sum(Amount) as Total from map_transation where year = {Year} and quarter = {Quarter} group by district order by Total desc limit 10")
        df1 = pd.DataFrame(mycursor.fetchall(), columns=['District', 'Transactions_Count','Total_Amount'])
        
        
        fig = px.pie(df1, values='Total_Amount',
                     names='District',
                     title='Top 10',
                     color_discrete_sequence=px.colors.sequential.Agsunset,
                     hover_data=['Transactions_Count'],
                     labels={'Transactions_Count':'Transactions_Count'})

        fig.update_traces(textposition='inside', textinfo='percent+label')
        st.plotly_chart(fig,use_container_width=True)
        
    
# top user info          
    if st.selectbox("**Type**", ("Transactions", "Users")) == "Users":
        col1,col2,col3 =st.columns([2,2,2],gap="small")
    with col1:
        st.markdown("### :violet[Brands]")
        if Year == 2022 and Quarter in [2,3,4]:
            st.error('The Data are inbetween 2022', icon="🚨")
        else:
            mycursor.execute(f"select brands, sum(count) as Total_Count, avg(percentage)*100 as Avg_Percentage from aggregated_user where year = {Year} and quarter = {Quarter} group by brands order by Total_Count desc limit 10")
            df2 = pd.DataFrame(mycursor.fetchall(), columns=['Brand', 'Total_Users','Avg_Percentage'])
            fig = px.bar(df2,
                         title='Top 10',
                         x="Total_Users",
                         y="Brand",
                         orientation='h',
                         color='Avg_Percentage',
                         color_continuous_scale=px.colors.sequential.Agsunset)
            st.plotly_chart(fig,use_container_width=True)
            df5 = pd.DataFrame(mycursor.fetchall(), columns=['State', 'Total_Users','Total_Appopens'])
            
    with col2:
        st.markdown("### :violet[State]")
        mycursor.execute(f"select state, sum(Registered_user) as Total_Users, sum(App_opens) as Total_Appopens from map_user where year = {Year} and quarter = {Quarter} group by state order by Total_Users desc limit 10")
        df3 = pd.DataFrame(mycursor.fetchall(), columns=['State', 'Total_Users','Total_Appopens'])
        fig = px.pie(df3, values='Total_Users',
                     names='State',
                     title='Top 10',
                     color_discrete_sequence=px.colors.sequential.Agsunset,
                     hover_data=['Total_Appopens'],
                     labels={'Total_Appopens':'Total_Appopens'})
        fig.update_traces(textposition='inside', textinfo='percent+label')
        st.plotly_chart(fig,use_container_width=True)
    
    with col3:
        st.markdown("### :violet[District]")
        mycursor.execute(f"select district, sum(Registered_User) as Total_Users, sum(app_opens) as Total_Appopens from map_user where year = {Year} and quarter = {Quarter} group by district order by Total_Users desc limit 10")
        df4 = pd.DataFrame(mycursor.fetchall(), columns=['District', 'Total_Users','Total_Appopens'])
        df4.Total_Users = df4.Total_Users.astype(float)
        fig = px.bar(df4,
                     title='Top 10',
                     x="Total_Users",
                     y="District",
                     orientation='h',
                     color='Total_Users',
                     color_continuous_scale=px.colors.sequential.Agsunset)
        
        
        st.plotly_chart(fig,use_container_width=True)
           
#user selected data with map        
if selected == "Explore Now":
    state_mapping = {
    'andaman-&-nicobar-islands': 'Andaman & Nicobar Island',
    'andhra-pradesh': 'Andhra Pradesh',
    'arunachal-pradesh': 'Arunanchal Pradesh',
    'assam': 'Assam',
    'bihar': 'Bihar',
    'chandigarh': 'Chandigarh',
    'chhattisgarh': 'Chhattisgarh',
    'dadra-&-nagar-haveli-&-daman-&-diu': 
    'Dadara & Nagar Havelli',
    'delhi': 'NCT of Delhi',
    'goa': 'Goa',
    'gujarat': 'Gujarat',
    'haryana': 'Haryana',
    'himachal-pradesh': 'Himachal Pradesh',
    'jammu-&-kashmir': 'Jammu & Kashmir',
    'jharkhand': 'Jharkhand',
    'karnataka': 'Karnataka',
    'kerala': 'Kerala',
    'ladakh': 'Ladakh',
    'lakshadweep': 'Lakshadweep',
    'madhya-pradesh': 'Madhya Pradesh',
    'maharashtra': 'Maharashtra',
    'manipur': 'Manipur',
    'meghalaya': 'Meghalaya',
    'mizoram': 'Mizoram',
    'nagaland': 'Nagaland',
    'odisha': 'Odisha',
    'puducherry': 'Puducherry',
    'punjab': 'Punjab',
    'rajasthan': 'Rajasthan',
    'sikkim': 'Sikkim',
    'tamil-nadu': 'Tamil Nadu',
    'telangana': 'Telangana',
    'tripura': 'Tripura',
    'uttar-pradesh': 'Uttar Pradesh',
    'uttarakhand': 'Uttarakhand',
    'west-bengal': 'West Bengal'
}
    
    
    st.markdown("## :violet[Overall State Data - Transactions Amount]")
    Year = st.slider("**Year**", min_value=2018, max_value=2022)
    Quarter = st.slider("Quarter", min_value=1, max_value=4)
    mycursor.execute(f"select state, sum(count) as Total_Transactions, sum(amount) as Total_amount from map_transation where year = {Year} and quarter = {Quarter} group by state order by state")
    df5 = pd.DataFrame(mycursor.fetchall(),columns= ['State', 'Total_Transactions', 'Total_amount'])
    
    df5.Total_Transactions = df5.Total_Transactions.astype(int)

# Apply the mapping
df5['State'] = df5['State'].map(state_mapping)

df5 = df5.drop(17)

india_states = json.load(open(r"Downloads/states_india.geojson"))

state_id_map ={}
for feature in india_states['features']:
    feature ['id'] = feature['properties']['state_code']
    state_id_map[feature['properties']['st_nm']] = feature['id']
    
df5['id'] = df5['State'].apply(lambda x: state_id_map[x])

selected_type = st.selectbox("**Type**", ("Total_amount", "Total_Transactions","Total_Userdata"), key="select_type")

if selected_type == "Total_amount":
    st.subheader(":violet[Total State's Transactions Amount Data]🌃")

    fig = px.choropleth(df5, 
                        locations='id', 
                        geojson=india_states, 
                        color='Total_amount',
                        hover_name='State',
                        hover_data=['Total_amount'])

    fig.update_geos(fitbounds='locations', visible=False)
    fig.update_layout(width=800, height=800)
    
    st.plotly_chart(fig, use_container_width=True)
    
if selected_type == "Total_Transactions":
    st.subheader(":violet[Total State's Transactions Count Data]🌃")

    fig = px.choropleth(df5, 
                        locations='id', 
                        geojson=india_states, 
                        color='Total_Transactions',
                        hover_name='State',
                        hover_data=['Total_Transactions'])

    fig.update_geos(fitbounds='locations', visible=False)
    fig.update_layout(width=800, height=800)
    
    st.plotly_chart(fig, use_container_width=True)
    
if selected_type == "Total_Userdata":
    
    mycursor.execute(f"select state, sum(Registered_user) as Total_Users, sum(App_opens) as Total_Appopens from map_user where year = {Year} and quarter = {Quarter} group by state order by state")
    df6 = pd.DataFrame(mycursor.fetchall(), columns=['State', 'Total_Users','Total_Appopens'])
    
    df6.Total_Appopens = df6.Total_Appopens.astype(float)
    
    
    # Apply the mapping
df6['State'] = df6['State'].map(state_mapping)

df6 = df.drop(17)

india_states = json.load(open(r"Downloads/states_india.geojson"))

state_id_map ={}
for feature in india_states['features']:
    feature ['id'] = feature['properties']['state_code']
    state_id_map[feature['properties']['st_nm']] = feature['id']
    
df6['id'] = df6['State'].apply(lambda x: state_id_map[x])

fig = px.choropleth(df, 
                        locations='id', 
                        geojson=india_states, 
                        color='Total_Appopens',
                        hover_name='State',
                        hover_data=['Total_Appopens'])

fig.update_geos(fitbounds='locations', visible=False)
fig.update_layout(width=800, height=800)
    
st.plotly_chart(fig, use_container_width=True)